### We run the optical from on 24CPU, it took around 8 minutes. You can generate it yourselves or use the already extracted values available in ./resources/

[info]
-- Apparently, there is a bug in some version of the OpenCV, and the inf magnitude is estimated ... Works with 4.10.82

In [ ]:
import torch
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import os
import cv2

def process(current_frame, previous_frame, motion_threshold=None):
    """
    Computes the degree of motion between the current and previous frames.

    Args:
        current_frame (np.ndarray): Current frame in grayscale.
        previous_frame (np.ndarray): Previous frame in grayscale.
        motion_threshold (float): Threshold to filter out low-motion noise.

    Returns:
        float: A single value representing the degree of motion in the frame.
    """
    if previous_frame is None:
        return {'score': None}

    # Calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(
        prev=previous_frame,
        next=current_frame,
        flow=None,
        pyr_scale=0.5,
        levels=3,
        winsize=15,
        iterations=3,
        poly_n=5,
        poly_sigma=1.2,
        flags=0
    )

    # Compute magnitude and angle of flow vectors
    magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=True)
    # magnitude = np.clip(magnitude, 0, 1e6)

    # Filter out small magnitudes (noise) using a threshold
    if motion_threshold is not None:
        significant_motion = magnitude[magnitude > motion_threshold]
    else:
        significant_motion = magnitude

    # Calculate the mean magnitude of significant motion
    motion_score = significant_motion.mean() if significant_motion.size > 0 else 0.0

    return {'score': motion_score}

In [ ]:
import torch
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import os
import cv2

def process(current_frame, previous_frame, motion_threshold=None):
    """
    Computes the degree of motion between the current and previous frames.

    Args:
        current_frame (np.ndarray): Current frame in grayscale.
        previous_frame (np.ndarray): Previous frame in grayscale.
        motion_threshold (float): Threshold to filter out low-motion noise.

    Returns:
        float: A single value representing the degree of motion in the frame.
    """
    if previous_frame is None:
        return {'score': None}

    # Calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(
        prev=previous_frame,
        next=current_frame,
        flow=None,
        pyr_scale=0.5,
        levels=3,
        winsize=15,
        iterations=3,
        poly_n=5,
        poly_sigma=1.2,
        flags=0
    )

    # Compute magnitude and angle of flow vectors
    magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=True)
    magnitude = np.clip(magnitude, 0, 1e19)

    # Filter out small magnitudes (noise) using a threshold
    if motion_threshold is not None:
        significant_motion = magnitude[magnitude > motion_threshold]
    else:
        significant_motion = magnitude

    # Calculate the mean magnitude of significant motion
    motion_score = significant_motion.mean() if significant_motion.size > 0 else 0.0

    return {'score': motion_score}

folder_path = "../../dataset/coool-benchmark/"
target_fps = 5

results = {}
for filename in tqdm(os.listdir(folder_path)):

    if not filename == 'video_0079.mp4':
        continue
    
    if filename.endswith(".mp4"):
        video_path = os.path.join(folder_path, filename)
        cap = cv2.VideoCapture(video_path)
        original_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_skip = int(original_fps / target_fps)

        # Initialize storage for this video
        previous_frame = None
        video_results = []
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % frame_skip == 0:
                # Convert the current frame to grayscale
                current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
                # Calculate optical flow between consecutive frames
                result = process(current_frame, previous_frame)
                result["frame"] = frame_count
                video_results.append(result)
                # Update the previous frame
                previous_frame = current_frame

            frame_count += 1

        cap.release()
        results[filename] = video_results
torch.save(results, f'optical_flow.pkl')

In [ ]:
import os
import cv2
import torch
from tqdm import tqdm
from joblib import Parallel, delayed

# Define the folder path and target FPS
FOLDER_PATH = "../../dataset/coool-benchmark/"  # <-- UPDATE THIS ONE !!!
RESULTS_FOLDER = "../../resources/optical-flow/"  # Folder to store results
RESULTS_FILE = os.path.join(RESULTS_FOLDER, "optical_flow.pkl")
target_fps = 5

# Ensure results folder exists
os.makedirs(RESULTS_FOLDER, exist_ok=True)

def process_frame_sequence(filename):
    """
    Process a single video file to compute optical flow at the target FPS.

    Args:
        filename (str): Name of the video file to process.

    Returns:
        dict: Dictionary containing video filename as the key and optical flow results as the value.
    """
    video_path = os.path.join(FOLDER_PATH, filename)
    if not os.path.exists(video_path):
        return {filename: None}  # Skip processing if the file doesn't exist

    cap = cv2.VideoCapture(video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip = int(original_fps / target_fps)

    previous_frame = None
    video_results = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            result = process(current_frame, previous_frame)  # Assuming process() is defined elsewhere
            result["frame"] = frame_count
            video_results.append(result)
            previous_frame = current_frame

        frame_count += 1

    cap.release()
    return {filename: video_results}

In [ ]:
# Get list of video files to process
video_files = [f for f in os.listdir(FOLDER_PATH) if f.endswith(".mp4")]

# Process videos in parallel using joblib.Parallel
results_list = Parallel(n_jobs=-1)(delayed(process_frame_sequence)(filename) for filename in tqdm(video_files))

# Combine results from all videos
results = {}
for result in results_list:
    results.update(result)

# Save results to file
torch.save(results, RESULTS_FILE)